# 数据预处理与特征工程

## 1、概述

### 1.1、数据预处理与特征工程

- 数据挖掘的五大流程：
    - 1. 获取数据
    - 2. **数据预处理**
    
        数据预处理是从数据中检测，纠正或删除损坏，不准确或不适用于模型的记录的过程
    可能面对的问题有：数据类型不同，比如有的是文字，有的是数字，有的含时间序列，有的连续，有的间断。
    也可能，数据的质量不行，有噪声，有异常，有缺失，数据出错，量纲不一，有重复，数据是偏态，数据量太
    大或太小
    
        数据预处理的目的：让数据适应模型，匹配模型的需求
    - 3. **特征工程**
        
        特征工程是将原始数据转换为更能代表预测模型的潜在问题的特征的过程，可以通过挑选最相关的特征，提取
    特征以及创造特征来实现。其中创造特征又经常以降维算法的方式实现。
    可能面对的问题有：特征之间有相关性，特征和标签无关，特征太多或太小，或者干脆就无法表现出应有的数
    据现象或无法展示数据的真实面貌
    
        特征工程的目的：1) 降低计算成本，2) 提升模型上限
    - 4. 建模，测试模型并预测出结果
    - 5. 上线，验证模型效果
    
### 1.2、sklearn中的数据预处理和特征工程

sklearn中包含众多数据预处理和特征工程相关的模块，虽然刚接触sklearn时，大家都会为其中包含的各种算法的
广度深度所震惊，但其实sklearn六大板块中有两块都是关于数据预处理和特征工程的，两个板块互相交互，为建
模之前的全部工程打下基础。

![](../imgs/sklearn_unit.png)

- 模块preprocessing：几乎包含数据预处理的所有内容
- 模块Impute：填补缺失值专用
- 模块feature_selection：包含特征选择的各种方法的实践
- 模块decomposition：包含降维算法


## 2、数据预处理 Preprocessing & Impute
### 2.1、数据无量纲化

在机器学习算法实践中，我们往往有着将不同规格的数据转换到同一规格，或不同分布的数据转换到某个特定分布
的需求，这种需求统称为将数据“无量纲化”。譬如梯度和矩阵为核心的算法中，譬如逻辑回归，支持向量机，神经
网络，无量纲化可以加快求解速度；而在距离类模型，譬如K近邻，K-Means聚类中，无量纲化可以帮我们提升模
型精度，避免某一个取值范围特别大的特征对距离计算造成影响。（一个特例是决策树和树的集成算法们，对决策
树我们不需要无量纲化，决策树可以把任意数据都处理得很好。）

数据的无量纲化可以是线性的，也可以是非线性的。线性的无量纲化包括**中心化（Zero-centered或者Meansubtraction）处理**和**缩放处理（Scale）**。中心化的本质是让所有记录减去一个固定值，即让数据样本数据平移到某个位置。缩放的本质是通过除以一个固定值，将数据固定在某个范围之中，取对数也算是一种缩放处理

- **preprocessing.MinMaxScaler**

    当数据(x)按照最小值中心化后，再按极差（最大值 - 最小值）缩放，数据移动了最小值个单位，并且会被收敛到
    [0,1]之间，而这个过程，就叫做**数据归一化**(Normalization，又称Min-Max Scaling)。注意，Normalization是归
    一化，不是正则化，真正的正则化是regularization，不是数据预处理的一种手段。归一化之后的数据服从正态分
    布，公式如下：

    ![](../imgs/min-max-scaling.png)

    在sklearn当中，我们使用**preprocessing.MinMaxScaler**来实现这个功能。MinMaxScaler有一个重要参数，
    feature_range，控制我们希望把数据压缩到的范围，默认是[0,1]。

In [115]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import Binarizer, KBinsDiscretizer
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

In [4]:
data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]

pd.DataFrame(data)

,0,1
0,-1.0,2
1,-0.5,6
2,0.0,10
3,1.0,18


In [15]:
# 实现归一化
scalar = MinMaxScaler()  # 实例化
scalar = scalar.fit(data)  #  fit，在这里本质是生成max(x)和min(x)
result = scalar.transform(data)  # 通过接口导出结果
result    # 两个看似不一样的结果归一化后一样

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [7]:
result_ = scalar.fit_transform(data)   # 训练和导出结果一步完成
result_

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [8]:
scalar.inverse_transform(result)   # 将归一化后的结果逆转

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

In [9]:
# 使用MinMaxScaler的参数feature_range实现将数据归一化到[0,1]以外的范围中
scalar = MinMaxScaler(feature_range=[5,10])
result = scalar.fit_transform(data)
result

array([[ 5.  ,  5.  ],
       [ 6.25,  6.25],
       [ 7.5 ,  7.5 ],
       [10.  , 10.  ]])

- **注意**：
    - 当X中的特征数量非常多的时候，fit会报错并表示，数据量太大了我计算不了
    - 此时使用**partial_fit**作为训练接口
    - scaler = scaler.partial_fit(data)

- **BONUS: 使用numpy来实现归一化**

In [14]:
X = np.array([[-1, 2], [-0.5, 6], [0, 10], [1, 18]])

# 归一化

X_nor = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_nor

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [16]:
# 逆转归一化
X_returned = X_nor * (X.max(axis=0) - X.min(axis=0)) + X.min(axis=0)
X_returned

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

- **preprocessing.StandardScaler**

    当数据(x)按均值(μ)中心化后，再按标准差(σ)缩放，数据就会服从为均值为0，方差为1的正态分布（即标准正态分布），而这个过程，就叫做**数据标准化**(Standardization，又称Z-score normalization)，公式如下：

![](../imgs/standardzation.png)


In [19]:
scalar = StandardScaler()  # 实例化
scalar.fit(data)           # fit，本质是生成均值和方差

StandardScaler(copy=True, with_mean=True, with_std=True)

In [20]:
# 查看均值的属性：mean_
scalar.mean_

array([-0.125,  9.   ])

In [21]:
# 查看方差的属性：var_
scalar.var_

array([ 0.546875, 35.      ])

In [22]:
# 通过接口导出结果
x_std = scalar.transform(data)

In [25]:
# 2列不同的值，标准化后返回了相同的值，分布一样
x_std

array([[-1.18321596, -1.18321596],
       [-0.50709255, -0.50709255],
       [ 0.16903085,  0.16903085],
       [ 1.52127766,  1.52127766]])

In [27]:
# 查看均值：mean()
x_std.mean()

0.0

In [28]:
# 查看标准差：std()
x_std.std()

1.0

In [29]:
# 一步到位
scalar.fit_transform(data)

array([[-1.18321596, -1.18321596],
       [-0.50709255, -0.50709255],
       [ 0.16903085,  0.16903085],
       [ 1.52127766,  1.52127766]])

In [30]:
# 逆转标准化
scalar.inverse_transform(x_std)

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

对于StandardScaler和MinMaxScaler来说，空值NaN会被当做是缺失值，在fit的时候忽略，在transform的时候保持缺失NaN的状态显示。并且，尽管去量纲化过程不是具体的算法，但在fit接口中，依然只允许导入至少二维数组，一维数组导入会报错。通常来说，我们输入的X会是我们的特征矩阵，现实案例中特征矩阵不太可能是一维所以不会存在这个问题。

- **StandardScaler和MinMaxScaler选哪个？**

看情况。大多数机器学习算法中，会选择StandardScaler来进行特征缩放，因为MinMaxScaler对异常值非常敏感。在PCA，聚类，逻辑回归，支持向量机，神经网络这些算法中，StandardScaler往往是最好的选择。

MinMaxScaler在不涉及距离度量、梯度、协方差计算以及数据需要被压缩到特定区间时使用广泛，比如数字图像处理中量化像素强度时，都会使用MinMaxScaler将数据压缩于[0,1]区间之中。

建议先试试看StandardScaler，效果不好换MinMaxScaler。

除了StandardScaler和MinMaxScaler之外，sklearn中也提供了各种其他缩放处理（中心化只需要一个pandas广播一下减去某个数就好了，因此sklearn不提供任何中心化功能）。比如，在希望压缩数据，却不影响数据的稀疏性时（不影响矩阵中取值为0的个数时），我们会使用MaxAbsScaler；在异常值多，噪声非常大时，我们可能会选用分位数来无量纲化，此时使用RobustScaler。更多详情请参考以下列表

![](../imgs/different_preprocess_data.webp)

### 2.2、缺失值

机器学习和数据挖掘中所使用的数据，永远不可能是完美的。很多特征，对于分析和建模来说意义非凡，但对于实际收集数据的人却不是如此，因此数据挖掘之中，常常会有重要的字段缺失值很多，但又不能舍弃字段的情况。因此，数据预处理中非常重要的一项就是处理缺失值。

In [10]:
# index_col=0：把第0列作为索引
data = pd.read_csv("../data/Taitanic/Narrativedata.csv",
                  index_col=0)
data.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,No
1,38.0,female,C,Yes
2,26.0,female,S,Yes
3,35.0,female,S,Yes
4,35.0,male,S,No


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
Age         714 non-null float64
Sex         891 non-null object
Embarked    889 non-null object
Survived    891 non-null object
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


在这里，我们使用从泰坦尼克号提取出来的数据，这个数据有三个特征，一个数值型，两个字符型，标签也是字符型。从这里开始，我们就使用这个数据给大家作为例子，让大家慢慢熟悉sklearn中数据预处理的各种方式。

- impute.SimpleImputer

`class sklearn.impute.SimpleImputer(missing_values=nan, strategy=’mean’, fill_value=None, verbose=0, copy=True)`

在讲解随机森林的案例时，我们用这个类和随机森林回归填补了缺失值，对比了不同的缺失值填补方式对数据的影响。这个类是专门用来填补缺失值的。它包括四个重要参数：

| 参数 | 含义&输入 |
| :------ | :----- |
| missing_values | 告诉SimpleImputer，数据中的缺失值长什么样，默认空值np.nan |
| strategy | 我们填补缺失值的策略，默认均值。<br>输入“mean”使用均值填补（仅对数值型特征可用）<br>输入“median"用中值填补（仅对数值型特征可用）<br>输入"most_frequent”用众数填补（对数值型和字符型特征都可用）<br>输入“constant"表示请参考参数“fill_value"中的值（对数值型和字符型特征都可用） |
| fill_value | 当参数startegy为”constant"的时候可用，可输入字符串或数字表示要填充的值，常用0 |
| copy | 默认为True，将创建特征矩阵的副本，反之则会将缺失值填补到原本的特征矩阵中去。 |


In [20]:
# 填补年龄

# data.loc[:, 'Age']取出来的是Series
# sklearn当中特征矩阵必须是二维
Age = data.loc[:, 'Age'].values.reshape(-1, 1)
Age[:10]

array([[22.],
       [38.],
       [26.],
       [35.],
       [35.],
       [nan],
       [54.],
       [ 2.],
       [27.],
       [14.]])

In [22]:
imp_mean = SimpleImputer()  # 实例化，默认均值填充
imp_median = SimpleImputer(strategy='median')   # 中位数填补
imp_0 = SimpleImputer(strategy='constant', fill_value=0)  # 用0填补

# fit_transform一步到位
imp_mean = imp_mean.fit_transform(Age)
imp_median = imp_median.fit_transform(Age)
imp_0 = imp_0.fit_transform(Age)

In [25]:
imp_mean[:10]

array([[22.        ],
       [38.        ],
       [26.        ],
       [35.        ],
       [35.        ],
       [29.69911765],
       [54.        ],
       [ 2.        ],
       [27.        ],
       [14.        ]])

In [26]:
imp_median[:10]

array([[22.],
       [38.],
       [26.],
       [35.],
       [35.],
       [28.],
       [54.],
       [ 2.],
       [27.],
       [14.]])

In [27]:
imp_0[:10]

array([[22.],
       [38.],
       [26.],
       [35.],
       [35.],
       [ 0.],
       [54.],
       [ 2.],
       [27.],
       [14.]])

In [31]:
# 在这里我们使用中位数填补Age
data.loc[:, 'Age'] = imp_median
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
Age         891 non-null float64
Sex         891 non-null object
Embarked    889 non-null object
Survived    891 non-null object
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


In [37]:
# 使用众数填补Embarked，如果现实中，有2条数据缺失完全可以删除这2条
Embarked = data.loc[:, 'Embarked'].values.reshape(-1, 1)
Embarked[55:65]

array([['S'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['C'],
       [nan],
       ['S'],
       ['S'],
       ['C']], dtype=object)

In [39]:
imp_mode = SimpleImputer(strategy='most_frequent')
data.loc[:, "Embarked"] = imp_mode.fit_transform(Embarked)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
Age         891 non-null float64
Sex         891 non-null object
Embarked    891 non-null object
Survived    891 non-null object
dtypes: float64(1), object(3)
memory usage: 74.8+ KB


- **BONUS：用Pandas和Numpy进行填补其实更加简单**

In [42]:
data_ = pd.read_csv("../data/Taitanic/Narrativedata.csv",
                  index_col=0)
data_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
Age         714 non-null float64
Sex         891 non-null object
Embarked    889 non-null object
Survived    891 non-null object
dtypes: float64(1), object(3)
memory usage: 34.8+ KB


In [44]:
#.fillna 在DataFrame里面直接进行填补
data_.loc[:, 'Age'] = data.loc[:, "Age"].fillna(data.loc[:, "Age"].median())
# .dropna(axis=0)删除所有有缺失值的行，.dropna(axis=1)删除所有有缺失值的列
# 参数inplace，为True表示在原数据集上进行修改，为False表示生成一个复制对象，不修改原数据，默认False
data_.dropna(axis=0, inplace=True)
data_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 4 columns):
Age         889 non-null float64
Sex         889 non-null object
Embarked    889 non-null object
Survived    889 non-null object
dtypes: float64(1), object(3)
memory usage: 34.7+ KB


### 2.3、处理分类型特征：编码与哑变量

在机器学习中，大多数算法，譬如逻辑回归，支持向量机SVM，k近邻算法等都只能够处理数值型数据，不能处理文字，在sklearn当中，除了专用来处理文字的算法，其他算法在fit的时候全部要求输入数组或矩阵，也不能够导入文字型数据（其实手写决策树和普斯贝叶斯可以处理文字，但是sklearn中规定必须导入数值型）。然而在现实中，许多标签和特征在数据收集完毕的时候，都不是以数字来表现的。比如说，学历的取值可以是["小学"，“初中”，“高中”，"大学"]，付费方式可能包含["支付宝"，“现金”，“微信”]等等。在这种情况下，为了让数据适应算法和库，我们必须将数据进行**编码**，即是说，**将文字型数据转换为数值型**。

- **preprocessing.LabelEncoder：标签专用，能够将分类转换为分类数值**

In [58]:
y = data.iloc[:, -1]   # 要输入的是标签，不是特征矩阵，所以运行一维

le = LabelEncoder()
le = le.fit(y)           # 导入数据
label = le.transform(y)  # 调取数据
# le.fit_transform(y) #也可以直接fit_transform一步到位
label[:20]               # 结果出现0,1,2
# le.inverse_transform(label) #使用inverse_transform可以逆转

array([0, 2, 2, 2, 0, 0, 0, 0, 2, 2, 1, 2, 0, 0, 0, 1, 0, 2, 0, 2])

In [113]:
le.classes_              # 属性.classes_查看标签中究竟有多少类别

array(['No', 'Unknown', 'Yes'], dtype=object)

In [59]:
data.iloc[:,-1] = label  # 赋值
data.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


In [61]:
# 简写(熟悉sklearn后都可以这样简写)
data.iloc[:, -1] = LabelEncoder().fit_transform(data.iloc[:, -1])
data.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


- **preprocessing.OrdinalEncoder：特征专用，能够将分类特征转换为分类数值**

In [63]:
data.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


In [76]:
data_ = data.copy()
# 接口categories_对应LabelEncoder的接口classes_，一模一样的功能
OrdinalEncoder().fit(data_.iloc[:, 1:-1]).categories_

[array(['female', 'male'], dtype=object), array(['C', 'Q', 'S'], dtype=object)]

In [80]:
data_.iloc[:, 1:-1] = OrdinalEncoder().fit_transform(data_.iloc[:, 1:-1])
data_.head()

,Age,Sex,Embarked,Survived
0,22.0,1.0,2.0,0
1,38.0,0.0,0.0,2
2,26.0,0.0,2.0,2
3,35.0,0.0,2.0,2
4,35.0,1.0,2.0,0


- **preprocessing.OneHotEncoder：独热编码，创建哑变量**

我们刚才已经用OrdinalEncoder把分类变量Sex和Embarked都转换成数字对应的类别了。在舱门Embarked这一列中，我们使用[0,1,2]代表了三个不同的舱门，然而这种转换是正确的吗？

我们来思考三种不同性质的分类数据：

1） 舱门（S，C，Q）

​ 三种取值S，C，Q是相互独立的，彼此之间完全没有联系，表达的是S≠C≠Q的概念。这是**名义变量**。

2） 学历（小学，初中，高中）

​ 三种取值不是完全独立的，我们可以明显看出，在性质上可以有高中>初中>小学这样的联系，学历有高低，但是学历取值之间却不是可以计算的，我们不能说小学 + 某个取值 = 初中。这是**有序变量**。

3） 体重（>45kg，>90kg，>135kg）

​ 各个取值之间有联系，且是可以互相计算的，比如120kg - 45kg = 90kg，分类之间可以通过数学计算互相转换。这是**有距变量**。

然而在对特征进行编码的时候，这三种分类数据都会被我们转换为[0,1,2]，这三个数字在算法看来，是连续且可以计算的，这三个数字相互不等，有大小，并且有着可以相加相乘的联系。所以算法会把舱门，学历这样的分类特征，都误会成是体重这样的分类特征。这是说，我们把分类转换成数字的时候，忽略了数字中自带的数学性质，所以给算法传达了一些不准确的信息，而这会影响我们的建模。

类别OrdinalEncoder可以用来处理有序变量，但对于名义变量，我们只有使用哑变量的方式来处理，才能够尽量向算法传达最准确的信息：

![](../imgs/onehot_encode.webp)

这样的变化，让算法能够彻底领悟，原来三个取值是没有可计算性质的，是“有你就没有我”的不等概念。在我们的数据中，性别和舱门，都是这样的名义变量。因此我们需要使用独热编码，将两个特征都转换为哑变量。

In [81]:
data.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


In [95]:
x = data.iloc[:, 1:-1]
enc = OneHotEncoder(categories='auto').fit(x)
# 一步到位
result = OneHotEncoder(categories='auto').fit_transform(x).toarray()
result
# 2个特征：年龄2个类别，舱门：3个，所以结果有5列

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

In [92]:
# 还原
pd.DataFrame(enc.inverse_transform(result))

,0,1
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S
...,...,...
886,male,S
887,female,S
888,female,S
889,male,C


In [96]:
enc.get_feature_names() # 返回热编码后稀疏矩阵的对应列的特征

array(['x0_female', 'x0_male', 'x1_C', 'x1_Q', 'x1_S'], dtype=object)

In [98]:
# axis=1,表示跨行进行合并，也就是将量表左右相连，如果是axis=0，就是将量表上下相连
newdata = pd.concat([data, pd.DataFrame(result)], axis=1)
newdata.head()

,Age,Sex,Embarked,Survived,0,1,2,3,4
0,22.0,male,S,0,0.0,1.0,0.0,0.0,1.0
1,38.0,female,C,2,1.0,0.0,1.0,0.0,0.0
2,26.0,female,S,2,1.0,0.0,0.0,0.0,1.0
3,35.0,female,S,2,1.0,0.0,0.0,0.0,1.0
4,35.0,male,S,0,0.0,1.0,0.0,0.0,1.0


In [99]:
newdata.drop(['Sex', 'Embarked'], inplace=True, axis=1)
newdata.head()

,Age,Survived,0,1,2,3,4
0,22.0,0,0.0,1.0,0.0,0.0,1.0
1,38.0,2,1.0,0.0,1.0,0.0,0.0
2,26.0,2,1.0,0.0,0.0,0.0,1.0
3,35.0,2,1.0,0.0,0.0,0.0,1.0
4,35.0,0,0.0,1.0,0.0,0.0,1.0


In [100]:
newdata.columns = ["Age","Survived","Female","Male","Embarked_C","Embarked_Q","Embarked_S"]
newdata.head()

,Age,Survived,Female,Male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,0,0.0,1.0,0.0,0.0,1.0
1,38.0,2,1.0,0.0,1.0,0.0,0.0
2,26.0,2,1.0,0.0,0.0,0.0,1.0
3,35.0,2,1.0,0.0,0.0,0.0,1.0
4,35.0,0,0.0,1.0,0.0,0.0,1.0


特征可以做哑变量，标签也可以吗？可以，使用类sklearn.preprocessing.LabelBinarizer可以对做哑变量，许多算法都可以处理多标签问题（比如说决策树），但是这样的做法在现实中不常见，因此我们在这里就不赘述了。

![](../imgs/Encoder_type.webp)

- **BONUS：数据类型以及常用的统计量**

![](../imgs/Encoder_data_type.png)

### 2.4、处理连续型特征：二值化与分段

- **sklearn.preprocessing.Binarizer**

根据阈值将数据二值化（将特征值设置为0或1），用于处理连续型变量。大于阈值的值映射为1，而小于或等于阈值的值映射为0。默认阈值为0时，特征中所有的正值都映射到1。二值化是对文本计数数据的常见操作，分析人员可以决定仅考虑某种现象的存在与否。它还可以用作考虑布尔随机变量的估计器的预处理步骤（例如，使用贝叶斯设置中的伯努利分布建模）。

In [110]:
data_2 = data.copy()
data_2.head()

,Age,Sex,Embarked,Survived
0,22.0,male,S,0
1,38.0,female,C,2
2,26.0,female,S,2
3,35.0,female,S,2
4,35.0,male,S,0


In [111]:
# 将年龄二值化
X = data_2.iloc[:, 0].values.reshape(-1,1)  # 类为特征专用，所以不能使用一维数组（所以特征专用的类都是）

bin_transform = Binarizer(threshold=30).fit_transform(X)
bin_transform[:5]

array([[0.],
       [1.],
       [0.],
       [1.],
       [1.]])

In [112]:
data_2.iloc[:, 0] = bin_transform
data_2.head()

,Age,Sex,Embarked,Survived
0,0.0,male,S,0
1,1.0,female,C,2
2,0.0,female,S,2
3,1.0,female,S,2
4,1.0,male,S,0


- **preprocessing.KBinsDiscretizer**

这是将连续型变量划分为分类变量的类，能够将连续型变量排序后按顺序分箱后编码。总共包含三个重要参数：

| 参数 | 含义&输入 |
| :------ | :----- |
| n_bins | 每个特征中分箱的个数，默认5，一次会被运用到所有导入的特征 |
| encode | <br>编码的方式，默认“onehot”<br>"onehot"：做哑变量，之后返回一个稀疏矩阵，每一列是一个特征中的一个类别，含有该<br>类别的样本表示为1，不含的表示为0（常用）<br>“ordinal”：每个特征的每个箱都被编码为一个整数，返回每一列是一个特征，每个特征下含<br>有不同整数编码的箱的矩阵（常用）<br>"onehot-dense"：做哑变量，之后返回一个密集数组。 |
| strategy | <br>用来定义箱宽的方式，默认"quantile"<br>"uniform"：表示等宽分箱，即每个特征中的每个箱的最大值之间的差为<br>(特征.max() - 特征.min())/(n_bins)<br>"quantile"：表示等位分箱，即每个特征中的每个箱内的样本数量都相同<br>"kmeans"：表示按聚类分箱，每个箱中的值到最近的一维k均值聚类的簇心得距离都相同 |



In [118]:
X = data.iloc[:, 0].values.reshape(-1, 1)

est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
est.fit_transform(X)[:10]

array([[0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [2.],
       [0.],
       [1.],
       [0.]])

In [121]:
# 查看转换后分的箱：变成了一列中的三箱
set(est.fit_transform(X).ravel())

{0.0, 1.0, 2.0}

In [125]:
est = KBinsDiscretizer(n_bins=3, encode='onehot', strategy='uniform')
# 查看转换后分的箱：变成了哑变量
pd.DataFrame(est.fit_transform(X).toarray())

,0,1,2
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
886,0.0,1.0,0.0
887,1.0,0.0,0.0
888,0.0,1.0,0.0
889,1.0,0.0,0.0
